In [1]:
import json
from StringIO import StringIO
import pandas as pd
import openpyxl
import pandas as pd
import os
import numpy as np
import datetime
import csv

In [2]:
import findspark
findspark.init()

from pyspark import SparkContext
sc = SparkContext(master="local[3]") 
from pyspark import SparkContext
from pyspark.sql import *
sqlContext = SQLContext(sc)

In [3]:
from pyspark.sql import Row
from pyspark.sql.types import *

In [4]:
listings = pd.read_csv('inside_airbnb/listings.csv')

In [5]:
special_events = pd.read_csv('Datasources/sdcounty_datasets/special_events_list_datasd.csv')

In [6]:
schema = StructType([StructField("id", IntegerType(), True),
                    StructField("longitude", DoubleType(), True),
                    StructField("latitude", DoubleType(), True)])

In [7]:
schema_two = StructType([StructField("event_id", IntegerType(), True),
                    StructField("longitude", DoubleType(), True),
                    StructField("latitude", DoubleType(), True)])

In [8]:
spql_listings_info = sqlContext.createDataFrame(listings[['id', 'longitude', 'latitude']], schema)

In [9]:
spql_special_events_info = sqlContext.createDataFrame(special_events[['event_id', 'longitude', 'latitude']].dropna(), schema_two)

In [10]:
spql_listings_info_rdd = spql_listings_info.rdd.map(lambda row:(row.id, row.longitude,row.latitude))

In [11]:
spql_special_events_info_rdd = spql_special_events_info.rdd.map(lambda row:(row.event_id, row.longitude,row.latitude))

In [12]:
spql_listings_and_events_info_rdd = spql_listings_info_rdd.cartesian(spql_special_events_info_rdd)

In [13]:
spql_listings_and_events_info_rdd = spql_listings_and_events_info_rdd.map(lambda x: (x[0][0], x[0][1], x[0][2], x[1][0], x[1][1], x[1][2]))

In [14]:
def distance_calculation(x):
        p_one = float(x[2])
        q_one = float(x[1])
        p_two = float(x[5])
        q_two = float(x[4])
        lon_diff = (q_one - q_two)*np.pi/180
        lat_diff = (p_one - p_two)*np.pi/180
        a = np.sin(lat_diff/2)**2 + np.cos(p_one*np.pi/180)*np.cos(p_two*np.pi/180)*(np.sin(lon_diff/2)**2)
        c = np.arctan2(np.sqrt(a), np.sqrt(1 - a))
        d = 6371.00*float(c)
        return tuple(list(x) + [d])

In [15]:
spql_listings_and_events_info_rdd = spql_listings_and_events_info_rdd.map(distance_calculation)

In [16]:
schema_three = StructType([StructField("id", IntegerType(), True),
                    StructField("longitude", DoubleType(), True),
                    StructField("latitude", DoubleType(), True)] + [StructField("event_id", IntegerType(), True),
                    StructField("event_longitude", DoubleType(), True),
                    StructField("event_latitude", DoubleType(), True), StructField("distance", DoubleType(), True)])

In [17]:
spql_listings_and_events_info_df = sqlContext.createDataFrame(spql_listings_and_events_info_rdd, schema_three)

In [18]:
spql_listings_and_events_info_rdd = spql_listings_and_events_info_df.rdd.map(lambda row: (row.id, row.longitude, row.latitude, row.event_id, row.event_longitude, row.event_latitude, row.distance))

In [19]:
closest_event_distance = spql_listings_and_events_info_rdd.map(lambda x: (x[0], x[-1])).reduceByKey(lambda x, y : min(x, y))

In [20]:
closest_event_distance = spql_listings_and_events_info_rdd.cartesian(closest_event_distance).filter(lambda x: x[0][-1] == x[1][-1]).map(lambda x: (x[0][0], x[0][-1], x[0][3]))

In [21]:
close_events_count_one = spql_listings_and_events_info_rdd.filter(lambda x: x[-1] <= 1).map(lambda x: (x[0], 1)).reduceByKey(lambda x, y : x + y)
close_events_count_three = spql_listings_and_events_info_rdd.filter(lambda x: x[-1] <= 3).map(lambda x: (x[0], 1)).reduceByKey(lambda x, y : x + y)
close_events_count_five = spql_listings_and_events_info_rdd.filter(lambda x: x[-1] <= 5).map(lambda x: (x[0], 1)).reduceByKey(lambda x, y : x + y)
close_events_count_ten = spql_listings_and_events_info_rdd.filter(lambda x: x[-1] <= 10).map(lambda x: (x[0], 1)).reduceByKey(lambda x, y : x + y)

In [22]:
close_events_count_sixteen = spql_listings_and_events_info_rdd.filter(lambda x: x[-1] <= 16).map(lambda x: (x[0], 1)).reduceByKey(lambda x, y : x + y)

In [89]:
count = 0

In [90]:
import datetime
today = datetime.date.today()
count+=1
filename = 'Datasources/listings_events/listings_events_' + str(today) + '_V' + str(count) + '.csv'

In [91]:
print filename

Datasources/listings_events/listings_events_2018-05-29_V1.csv


In [26]:
df1 = close_events_count_one.toDF()

In [27]:
df2 = close_events_count_three.toDF()

In [28]:
df3 = close_events_count_five.toDF()

In [29]:
#df4 = close_events_count_ten.toDF()

In [30]:
df4 = closest_event_distance.toDF()

In [31]:
df1 = df1.toPandas()

In [32]:
df1 = df1.rename(columns={'_1':'listing_id', '_2':'event_count_1km'})

In [33]:
df2 = df2.toPandas()

In [34]:
df2 = df2.rename(columns={'_1':'listing_id', '_2':'event_count_3km'})

In [35]:
df3 = df3.toPandas()

In [36]:
df3 = df3.rename(columns={'_1':'listing_id', '_2':'event_count_5km'})

In [37]:
#df4 = df4.toPandas()

In [38]:
#df4 = df4.rename(columns={'_1':'listing_id', '_2':'event_count_10km'})

In [39]:
df4 = df4.toPandas()

In [40]:
df4 = df4.rename(columns={'_1':'listing_id', '_2':'distance_to_closest_event_km', '_3': 'event_id_of_closest_distance'})

In [41]:
t1 = df1.merge(df2, how='outer', on='listing_id')

In [94]:
t2 = t1.merge(df3, how='outer', on='listing_id')
t3 = t2.merge(df4[['listing_id', 'distance_to_closest_event_km']], how='outer', on='listing_id')

In [99]:
t3 = t3.fillna(0)

In [101]:
t3_iter = t3.drop_duplicates()

In [103]:
t3_iter.index = range(len(t3_iter))

In [108]:
t3_iter['count_of_events_at_min_distance'] = 0

for idx in t3_iter.index:
    a = t3_iter['listing_id'][idx].item()
    b = len(t3[t3['listing_id'] == a])
    t3_iter.iloc[idx, 5] = b

C:\Users\sanka\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\sanka\Anaconda2\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [110]:
t3 = t3_iter

In [115]:
for idx in t3.index:
    a = t3['event_count_3km'][idx]
    b = t3['distance_to_closest_event_km'][idx]
    c = t3['count_of_events_at_min_distance'][idx]
    
    if (a == 0) and (1 < b) and (b < 3):
        t3.iloc[idx, 2] = c



In [118]:
#t4 = t3.merge(df5, how='inner', on='listing_id')

In [119]:
t3.head()

,listing_id,event_count_1km,event_count_3km,event_count_5km,distance_to_closest_event_km,count_of_events_at_min_distance
0,5179392,34.0,94.0,343.0,0.977183,34
1,7118856,146.0,431.0,962.0,0.080749,50
2,12953610,144.0,860.0,1625.0,0.108984,1
3,13711374,465.0,1421.0,1934.0,0.091013,7
4,13659480,1.0,53.0,1592.0,0.970600,1


In [121]:
t3.to_csv(filename,index=False)

In [122]:
t3.head()

,listing_id,event_count_1km,event_count_3km,event_count_5km,distance_to_closest_event_km,count_of_events_at_min_distance
0,5179392,34.0,94.0,343.0,0.977183,34
1,7118856,146.0,431.0,962.0,0.080749,50
2,12953610,144.0,860.0,1625.0,0.108984,1
3,13711374,465.0,1421.0,1934.0,0.091013,7
4,13659480,1.0,53.0,1592.0,0.970600,1


In [ ]:
#sc.stop()

In [123]:
len(t3)

6608